In [ ]:
# First Part:
# 1 - The problem I chose is that if a person wanted to open a restaurant in New York City, which place would be most ideal for him and his business.
# Obviously this problem targets business owners in NYC who would like to know what factors determine which place would be suitable for their business
# to thrive.

In [ ]:
# 2 - The data I will be using is the Foursquare location data.
# I will be using this data to view and determine which area in NYC is best for the restaurant, by looking at different factors which include:
# The shops and restaurants that are open near the area, the location of the area and whether its visited by many people or not,
# the rating of nearby shops and their popularity, whether the restaurant would be a unique addition to the area or not.